In [1]:
import os
import sys
import cv2 
import numpy as np
import torch
import torch.nn as nn
import time
import torch.nn.functional as F

In [2]:
sys.path.append("../10.mc_utils/")

In [3]:
from common.mview_merge_kp3d import compute_rigid_rotation,fix_missed_pts
from common.pose_solver import PoseSolver
from common.one_eurio_filter import OneEuroFilter

from wrappers.wrapper_mqtt_unity import WrapperMqttUnity3D
from wrappers.pipeline_config import CFG

In [4]:
pose_solver = PoseSolver(CFG.SOLVER.path)
traj_offset=np.array([[0.,1.5,0.]])## this trajectory is useless, just for local view

In [5]:
def load_kp3ds_to_torch(fname):
    assert os.path.exists(fname)
    raw_data = np.load(fname)
    #step1.load pose 3D
    gt_kp3ds = raw_data["kp3ds"]
    torch_gt_kp3ds = torch.from_numpy(gt_kp3ds).type(torch.float32)
    
    #step2.load trajectory
    trajs = raw_data["trajs"]

    #step3.load confidences
    confs = torch.from_numpy(raw_data["x2ds"][...,-1]).type(torch.float32)
    return [torch_gt_kp3ds,trajs,confs]

In [6]:
#####
## step1.load data from 2 views
## NOTE: Since I have no time, I didn't sync frame by frame.
##       Be sure each frame is synced!!!
##       A is main view
#####
torch_gt_kp3ds_A,traj_A,conf_A = load_kp3ds_to_torch("E:/20200110-area51/shotput_cap_20190227_110117_c400244/processed_3d.npz")
torch_gt_kp3ds_B,traj_B,conf_B = load_kp3ds_to_torch("E:/20200110-area51/shotput_cap_20190227_110117_c400246/processed_3d.npz")

In [7]:
###
## step1.pick frames
##       you can write a frame sync algorithm to sync frames here.
##       I simpley keep the short frames, and sync from the frame-0
###
N = min(torch_gt_kp3ds_A.shape[0],torch_gt_kp3ds_B.shape[0])
## one simple way to sync is to use set to compute mutal frame indexs


#step2.results list. and append the first pose from A
result_list = []
result_list.append(torch_gt_kp3ds_A[0])
body_euro_filter = None

#step3. merging poses
for i in range(1,N):
    s_A = torch_gt_kp3ds_A[i]
    s_B = torch_gt_kp3ds_B[i]
    s_prev = result_list[i-1]
    
    #step1.R mat
    ##  From B rotate to A
    R = compute_rigid_rotation(s_B,s_A)
    s_BA = torch.matmul(R,s_B.reshape(-1,3).t()).t()
    ## assemble to batch
    batch_skels = torch.stack([s_A,s_BA])#2x17x3
    batch_confs = torch.stack([conf_A[i],conf_B[i]])#2x17
    
    #step2.previous distances
    dists = torch.norm(batch_skels-s_prev.unsqueeze(0),dim=-1)#2x17 distances
    dists = torch.abs(torch.log10(dists+1e-8))*batch_confs#2x17
    
    #step3.compute weights
    weights = F.softmax(dists,dim=0)#shape is 2x17
    weights = weights.unsqueeze(-1)
    
    #step4.merge
    merged_skel = (weights*batch_skels).sum(0)
    
    #step5.smoothing
    ### NOTE, you can comment here off.
    ###
    if body_euro_filter is None:
        body_euro_filter = OneEuroFilter(i,
                                    merged_skel,
                                    min_cutoff=0.05,
                                    beta=0.03,
                                    d_cutoff = 0.01)
    merged_skel = body_euro_filter(i,merged_skel)
    
    #step6.save results.
    result_list.append(merged_skel)
merged_kp3ds = torch.stack(result_list)
merged_kp3ds = merged_kp3ds.numpy()

####
## TODO:
## add your saving record code here!!
####

In [8]:
#########################
## step2.get Amats
#########################
Amats_merged,_ = pose_solver.push(kp3ds=  merged_kp3ds.reshape(-1,51),
                                  traj = -traj_A[:N] + np.array([[0.,1.5,0.]]))

Amats_A,_ = pose_solver.push(kp3ds=  torch_gt_kp3ds_A[:N].reshape(-1,51),
                             traj = -traj_A[:N] + np.array([[2.0,1.5,0.]]))

batch_Amats = np.array([Amats_merged,Amats_A])
batch_ids = [0,1]

In [9]:
#step4.play
wrapper_unity3d = WrapperMqttUnity3D()
for i in range(N):
    wrapper_unity3d(batch_ids,batch_Amats[:,i])
    sys.stdout.write("\r>>>Frame[{}]<<<<<".format(i))
    time.sleep(1.0/60)

>>>Frame[1193]<<<<<